## Análisis inicial

In [34]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

# Para tokenización sin depender de 'punkt'
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from wordcloud import WordCloud

# Para análisis de sentimiento con VADER
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Para extracción de tópicos con LDA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Configuración de estilo para los gráficos
sns.set(style="whitegrid", context="talk")

# Habilitar la visualización inline en Jupyter Notebook
%matplotlib inline

In [35]:
df = pd.read_csv("data/fifa_world_cup_2022_tweets.csv")

In [36]:
print(df.head())

   Unnamed: 0               Date Created  Number of Likes  \
0           0  2022-11-20 23:59:21+00:00                4   
1           1  2022-11-20 23:59:01+00:00                3   
2           2  2022-11-20 23:58:41+00:00                1   
3           3  2022-11-20 23:58:33+00:00                1   
4           4  2022-11-20 23:58:28+00:00                0   

       Source of Tweet                                              Tweet  \
0      Twitter Web App  What are we drinking today @TucanTribe \n@MadB...   
1   Twitter for iPhone  Amazing @CanadaSoccerEN  #WorldCup2022 launch ...   
2   Twitter for iPhone  Worth reading while watching #WorldCup2022 htt...   
3      Twitter Web App  Golden Maknae shinning bright\n\nhttps://t.co/...   
4  Twitter for Android  If the BBC cares so much about human rights, h...   

  Sentiment  
0   neutral  
1  positive  
2  positive  
3  positive  
4  negative  


In [38]:
df.columns = df.columns.str.strip()
print(df.columns)

Index(['Unnamed: 0', 'Date Created', 'Number of Likes', 'Source of Tweet',
       'Tweet', 'Sentiment'],
      dtype='object')


In [39]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22524 entries, 0 to 22523
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       22524 non-null  int64 
 1   Date Created     22524 non-null  object
 2   Number of Likes  22524 non-null  int64 
 3   Source of Tweet  22524 non-null  object
 4   Tweet            22524 non-null  object
 5   Sentiment        22524 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.0+ MB
None


In [40]:
print(df.describe())

         Unnamed: 0  Number of Likes
count  22524.000000     22524.000000
mean   11261.500000        23.822856
std     6502.263068      2128.018705
min        0.000000         0.000000
25%     5630.750000         0.000000
50%    11261.500000         0.000000
75%    16892.250000         2.000000
max    22523.000000    316867.000000


## Preprocesamiento del texto

In [41]:
# Configurar las stopwords (en inglés; cambia a 'spanish' si es necesario)
stop_words = set(stopwords.words('english'))

# Crear un tokenizador que extraiga secuencias de caracteres alfanuméricos (no depende de punkt)
tokenizer = RegexpTokenizer(r'\w+')

def limpiar_texto(texto):
    # Convertir a minúsculas
    texto = texto.lower()
    # Eliminar URLs
    texto = re.sub(r'http\S+|www\S+|https\S+', '', texto, flags=re.MULTILINE)
    # Eliminar menciones (@usuario) y el símbolo de hashtag (se conserva la palabra)
    texto = re.sub(r'\@\w+', '', texto)
    texto = re.sub(r'\#', '', texto)
    # Eliminar caracteres especiales y números, dejando solo letras y espacios
    texto = re.sub(r'[^a-z\s]', '', texto)
    # Tokenizar usando RegexpTokenizer
    tokens = tokenizer.tokenize(texto)
    # Eliminar stopwords
    tokens = [token for token in tokens if token not in stop_words]
    return " ".join(tokens)

# Convertir la columna 'Tweet' a string (por si hay valores no-string)
df['Tweet'] = df['Tweet'].astype(str)

# Aplicar la función de limpieza a la columna "Tweet"
df['texto_limpio'] = df['Tweet'].apply(limpiar_texto)

# Mostrar algunos ejemplos comparando el tweet original y el limpio
print("\nEjemplos de preprocesamiento:")
print(df[['Tweet', 'texto_limpio']].head())


Ejemplos de preprocesamiento:
                                               Tweet  \
0  What are we drinking today @TucanTribe \n@MadB...   
1  Amazing @CanadaSoccerEN  #WorldCup2022 launch ...   
2  Worth reading while watching #WorldCup2022 htt...   
3  Golden Maknae shinning bright\n\nhttps://t.co/...   
4  If the BBC cares so much about human rights, h...   

                                        texto_limpio  
0                            drinking today worldcup  
1  amazing worldcup launch video shows much face ...  
2                    worth reading watching worldcup  
3  golden maknae shinning bright jeonjungkook jun...  
4  bbc cares much human rights homosexual rights ...  


## Análisis Exploratorio de Datos (EDA)